In [1]:
# Import get_engine function from sql_functions.py and set it to a variable called engine
# Import all necessary libraries
import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import * # package for unzipping zip files
import sqlalchemy
from sql_functions import get_engine 

In [2]:
# Import the Information about the schema & table name
schema = 'capstone_wildfire' 
table_name = 'combined_weather_data_v4'
engine = get_engine()

# Get fire_data as WF & weather_data as DF

In [3]:
# SQL-Abfrage, um Daten aus der Tabelle zu laden
sql_query = "SELECT * FROM capstone_wildfire.combined_weather_data_v4"

# Daten aus der Datenbank abrufen und in ein DataFrame laden
df = pd.read_sql_query(sql_query, engine)

# DataFrame anzeigen
print(df)

             date  average_temp_c  minimum_temp_c  maximum_temp_c  \
0      1995-01-10             NaN             NaN             NaN   
1      1995-05-24             NaN             NaN             NaN   
2      1995-12-13             NaN             NaN             NaN   
3      1995-03-11             NaN             NaN             NaN   
4      1995-10-15             NaN             NaN             NaN   
...           ...             ...             ...             ...   
550409 2010-10-15            27.7            22.2            33.9   
550410 2010-10-16            27.3            21.7            33.3   
550411 2010-10-17            27.2            21.7            32.8   
550412 2010-10-18            26.1            22.8            30.6   
550413 2010-10-19            25.1            21.1            31.1   

        precipitation_mm  snow_mm  wind_direction_deg  windspeed_kmh  \
0                    NaN      NaN                 NaN            NaN   
1                    NaN   

In [4]:
import pandas as pd

# CSV Files aus weather_and_wildfires.ipynb laden
wf = pd.read_csv('wild_fire_df.csv')

In [11]:
wf['combined_control_date'] = pd.to_datetime(wf['combined_control_date'])
wf['combined_discovery_date'] = pd.to_datetime(wf['combined_discovery_date'])

In [12]:
# # SQL-Abfrage, um Daten aus der Tabelle zu laden
# sql_query_wf = "SELECT * FROM capstone_wildfire.fires_data_v1_rdy_to_clean_rows"

# # Daten aus der Datenbank abrufen und in ein DataFrame laden
# wf = pd.read_sql_query(sql_query_wf, engine)

# # DataFrame anzeigen
# wf

To perform the statistical analysis on the given datasets using numpy, you can follow these steps:
Step 1: Import the required libraries
import numpy done


In [13]:
from scipy.stats import pearsonr, spearmanr, linregress, f_oneway


In [14]:
# Assuming 'date' column in df is not datetime, convert it to datetime
df['date'] = pd.to_datetime(df['date'])

In [15]:
# Group 'df' data by 'date' and calculate the daily average temperature
df_avg_temp = df.groupby(df['date'].dt.date)['average_temp_c'].mean().reset_index()

In [16]:
# Select the 'average_temp_c' column from the weather dataset
df_avg_temp = df_avg_temp[['date', 'average_temp_c']]

In [17]:
# Create a new column 'day' by extracting the day from the date
wf['day'] = wf['combined_discovery_date'].dt.date


In [18]:
# Count the number of unique wildfires discovered on each day
wf_agg = wf.groupby('day')['unique_id'].nunique().reset_index()
wf_agg.columns = ['Day', 'Number of Wildfires']


In [19]:
# Merge daily average temperature and wildfire data on the 'Day' column
merged_data = pd.merge(df_avg_temp, wf_agg, left_on='date', right_on='Day')

#This code will give you one 'weather data point' per day with the daily average temperature 
#for all 71 weather stations and the corresponding number of wildfires discovered on each day. 
#This merged data can then be used for Pearson Correlation analysis.

- wetterstationen, 71 insgesamt, 71 wetterstionen  seperat berechnet (das ist das problem )

In [21]:
# Remove rows with missing values (NaN) in either column
merged_data = merged_data.dropna(subset=['average_temp_c', 'Number of Wildfires'])

VISUALISIERUNG THE RESULT:

In [22]:
import matplotlib.pyplot as plt

In [23]:
# Assuming you have calculated the correlation coefficient and p-value
corr_coefficient = 0.297  
p_value = 1.216

Here's the code to create a grouped bar plot that displays both the correlation coefficient and the p-value:

VISUALIZATION OF THE # Calculate the Pearson correlation coefficient and p-value
#corr_coefficient, p_value = stats.pearsonr(merged_data['average_temp_c'], merged_data['Number of Wildfires'])

# diese berechnung als plot , scata plot


To visualize the Pearson correlation coefficient and p-value using a scatter plot in Python, you can create a scatter plot with the correlation coefficient on one axis and the negative logarithm (base 10) of the p-value on the other axis. This is a common approach to visualize correlation strength and statistical significance. Here's how you can do it.
In this code, we calculate the negative logarithm (base 10) of the p-value to visualize its significance more effectively. The scatter plot shows the relationship between the correlation coefficient and the significance of the correlation.

In [24]:
# Creating a Scatterplot
plt.figure(figsize=(10, 6))
x = merged_data['average_temp_c']
y = merged_data['Number of Wildfires']

# Creating the regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
line = slope * x + intercept

plt.scatter(x, y, label='Data Points', alpha=0.5)
plt.plot(x, line, color='red', label=f'Regression Line (R-squared={r_value**2:.2f})')
plt.title('Scatter Plot of Average Temperature vs. Number of Wildfires (1995-2015)')
plt.xlabel('Average Temperature (°C)')
plt.ylabel('Number of Wildfires')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'stats' is not defined

<Figure size 1000x600 with 0 Axes>

NEW DIAGRAM 2.10.23


In [ ]:

# Create new DataFrame from merged_df
average_temp_c_rounded = merged_data[['date', 'average_temp_c', 'Number of Wildfires']] 

# Round temperatures to nearest integer
average_temp_c_rounded['average_temp_c_rounded'] = average_temp_c_rounded['average_temp_c'].round().astype(int)

# Groupby rounded temperature and sum fire counts
average_temp_c_rounded = average_temp_c_rounded.groupby('average_temp_c_rounded')['Number of Wildfires'].sum().reset_index()

# Rename columns
average_temp_c_rounded.rename(columns={
    'average_temp_c_rounded': 'temp_c', 
    'Number of Wildfires': 'total_fires'
}, inplace=True)
average_temp_c_rounded

In [ ]:


# Streudiagramm mit einer Regressionslinie erstellen
plt.figure(figsize=(10, 6))
x = average_temp_c_rounded['temp_c']
y = average_temp_c_rounded['total_fires']

# Berechnung der Regressionslinie
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
line = slope * x + intercept

plt.scatter(x, y, label='Total fire', alpha=0.5)
plt.plot(x, line, color='red', label=f'Regression Line (R-squared={r_value**2:.2f})')
plt.title('Scatter Plot of Average Temperature vs. Number of Wildfires (1995-2015)')
plt.xlabel('Average Temperature (°C)')
plt.ylabel('Number of Wildfires')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Create new DataFrame from merged_df
average_temp_c_rounded = merged_data[['date', 'average_temp_c', 'Number of Wildfires']] 

# Round temperatures to nearest integer
average_temp_c_rounded['average_temp_c_rounded'] = average_temp_c_rounded['average_temp_c'].round().astype(int)

# Groupby rounded temperature and sum fire counts
average_temp_c_rounded = average_temp_c_rounded.groupby('average_temp_c_rounded')['Number of Wildfires'].sum().reset_index()

# Rename columns
average_temp_c_rounded.rename(columns={
    'average_temp_c_rounded': 'temp_c', 
    'Number of Wildfires': 'total_fires'
}, inplace=True)

# Streudiagramm mit einer Regressionslinie erstellen
plt.figure(figsize=(10, 6))
x = average_temp_c_rounded['temp_c']
y = average_temp_c_rounded['total_fires']

# Berechnung der Regressionslinie
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
line = slope * x + intercept

# Calculate Pearson Correlation Coefficient and P-Value
pearson_corr_coeff = r_value
p_value = p_value

plt.scatter(x, y, label=f'Data Points', alpha=0.5)
plt.plot(x, line, color='red', label=f'Regression Line (R-squared={r_value**2:.2f})')
plt.title('Scatter Plot of Average Temperature vs. Number of Wildfires (1995-2015)')
plt.xlabel('Average Temperature (°C)')
plt.ylabel('Number of Wildfires')

# Annotate the Pearson Correlation Coefficient and P-Value
plt.annotate(f'Pearson Correlation Coefficient: {pearson_corr_coeff:.4f}', (0.1, 0.85), xycoords='axes fraction', fontsize=12, color='green')
plt.annotate(f'P-Value: {p_value:.4e}', (0.1, 0.80), xycoords='axes fraction', fontsize=12, color='blue')

plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Create temperature blocks
temperature_blocks = [-5, 5, 15, 25]

# Categorize average temperatures into blocks
merged_data['Temp Block'] = pd.cut(merged_data['average_temp_c'], bins=temperature_blocks)

# Calculate the number of wildfires in each temperature block
block_counts = merged_data.groupby('Temp Block')['Number of Wildfires'].sum()

# Bar chart
plt.figure(figsize=(10, 6))
block_counts.plot(kind='bar', color='skyblue', alpha=0.7)
plt.title('Number of Wildfires vs. Average Temperature Blocks (1995-2015)')
plt.xlabel('Average Temperature Blocks (°C)')
plt.ylabel('Number of Wildfires')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add labels to the bars
for x, y in enumerate(block_counts):
    plt.text(x, y + 10, str(y), ha='center')

plt.show()
